In [13]:
import pandas as pd
import requests
import numpy as np
import time
import requests
import bs4
import json
from pymongo.errors import DuplicateKeyError, CollectionInvalid
from pymongo import MongoClient

In [17]:
client = MongoClient()
# Initiate Database
db = client['nyt_articles']
# Initiate Table
tab_biden = db['table_biden']
#tab_biden.insert({'biden': 'cool'})

db_comments = client['nyt_comments']
tab_biden = db_comments['table_biden']
db_comments.tab_biden.insert({'biden': 'cool'})

ObjectId('5601af151f80b40ad9d96444')

In [18]:
db = client['nyt_articles']
tab_carson = db['table_carson']
tab_carson.insert({'carson': 'cool'})

db_comments = client['nyt_comments']
tab_carson = db_comments['table_carson']
db_comments.tab_carson.insert({'carson': 'cool'})

ObjectId('5601af151f80b40ad9d96446')

In [6]:
# from nytimesarticle import articleAPI
# api = articleAPI('74c73309c1052e6aa1785df7cd5cef8c:9:69947183')

In [1]:
# for i in range(100):
#     articles = api.search(q = 'Joe Biden', page = i)
#     print i
#     for y in range(len(articles['response']['docs'])):
#         new_data = articles['response']['docs'][y]
#         new_data['_id'] = articles['response']['docs'][y]['_id']
#         try:
#             db.table_biden.insert(new_data)
#         except DuplicateKeyError:
#             print 'Peggy doesnt know why its a duplicate.'

In [8]:
NYT_URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
API_KEY = '74c73309c1052e6aa1785df7cd5cef8c:9:69947183'
import datetime as dt

In [9]:
# Query the NYT API once
def single_query(link, payload):
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print 'WARNING', response.status_code
    else:
        return response.json()

# Determine if the results are more than 100 pages
def more_than_100_pages(total_page):
    if total_page > 100:
        pages_left = min(total_page - 100, 100)
        return 100, pages_left, True
    else:
        return total_page, 0, False

# Looping through the pages give the number of pages
def loop_through_pages(total_pages, link, payload, table):
    for i in range(total_pages):
        if i % 50 == 0:
            print ' || Page ', i
        payload['page'] = str(i)
        content = single_query(link, payload)
        meta_lst = content['response']['docs']

        for meta in meta_lst:
            try:
                table.insert(meta)
            except DuplicateKeyError:
                print 'DUPS!'


# Scrape the meta data (link to article and put it into Mongo)
def scrape_meta(days=1):

    # The basic parameters for the NYT API
    link = NYT_URL
    payload = {'api-key': API_KEY, 'q' : 'Joe Biden', 'sort': 'newest'}

    today = dt.datetime(2015, 9, 22)
    yesterday = dt.datetime(2003, 9, 22)
    
    for day in range(days):
        payload['end_date'] = today.strftime("%Y%m%d")
        #yesterday = dt.date.today() - dt.timedelta(1)
        half_day = today - dt.timedelta(hours=12)
        payload['begin_date'] = yesterday.strftime("%Y%m%d")
        print payload
        print 'Scraping period: %s - %s ' % (str(yesterday), str(today))

        today -= dt.timedelta(days=2)
        
        content = single_query(link, payload)
        hits = content['response']['meta']['hits']
        total_pages = (hits / 10) + 1
        print 'HITS', hits

        newest_sort_pages, oldest_sort_pages, grt_100 = more_than_100_pages(total_pages)

        if grt_100:
            new_payload = payload.copy()
            old_payload = payload.copy()
            new_payload['sort']= 'newest'
            old_payload['sort'] = 'oldest'

        loop_through_pages(newest_sort_pages, link, new_payload, db.table_biden)
        loop_through_pages(oldest_sort_pages, link, old_payload, db.table_biden)

In [10]:
scrape_meta(days=1)

{'q': 'Joe Biden', 'sort': 'newest', 'begin_date': '20030922', 'api-key': '74c73309c1052e6aa1785df7cd5cef8c:9:69947183', 'end_date': '20150922'}
Scraping period: 2003-09-22 00:00:00 - 2015-09-22 00:00:00 
HITS 3431
 || Page  0
 || Page  50
 || Page  0
 || Page  50


In [11]:
# Query the NYT API once
def single_query(link, payload):
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print 'WARNING', response.status_code
    else:
        return response.json()

# Determine if the results are more than 100 pages
def more_than_100_pages(total_page):
    if total_page > 100:
        pages_left = min(total_page - 100, 100)
        return 100, pages_left, True
    else:
        return total_page, 0, False

# Looping through the pages give the number of pages
def loop_through_pages(total_pages, link, payload, table):
    for i in range(total_pages):
        if i % 50 == 0:
            print ' || Page ', i
        payload['page'] = str(i)
        content = single_query(link, payload)
        meta_lst = content['response']['docs']

        for meta in meta_lst:
            try:
                table.insert(meta)
            except DuplicateKeyError:
                print 'DUPS!'


# Scrape the meta data (link to article and put it into Mongo)
def scrape_meta(days=1):

    # The basic parameters for the NYT API
    link = NYT_URL
    payload = {'api-key': API_KEY, 'q' : 'Ben Carson', 'sort': 'newest'}

    today = dt.datetime(2015, 9, 22)
    yesterday = dt.datetime(2003, 9, 22)
    
    for day in range(days):
        payload['end_date'] = today.strftime("%Y%m%d")
        #yesterday = dt.date.today() - dt.timedelta(1)
        half_day = today - dt.timedelta(hours=12)
        payload['begin_date'] = yesterday.strftime("%Y%m%d")
        print payload
        print 'Scraping period: %s - %s ' % (str(yesterday), str(today))

        today -= dt.timedelta(days=2)
        
        content = single_query(link, payload)
        hits = content['response']['meta']['hits']
        total_pages = (hits / 10) + 1
        print 'HITS', hits

        newest_sort_pages, oldest_sort_pages, grt_100 = more_than_100_pages(total_pages)

        if grt_100:
            new_payload = payload.copy()
            old_payload = payload.copy()
            new_payload['sort']= 'newest'
            old_payload['sort'] = 'oldest'

        loop_through_pages(newest_sort_pages, link, new_payload, db.table_carson)
        loop_through_pages(oldest_sort_pages, link, old_payload, db.table_carson)

In [12]:
scrape_meta(days=1)

{'q': 'Ben Carson', 'sort': 'newest', 'begin_date': '20030922', 'api-key': '74c73309c1052e6aa1785df7cd5cef8c:9:69947183', 'end_date': '20150922'}
Scraping period: 2003-09-22 00:00:00 - 2015-09-22 00:00:00 
HITS 1189
 || Page  0
 || Page  50
 || Page  0
DUPS!


In [15]:
links = db.table_biden.find({},{'web_url': 1, '_id' : 0})
links_biden = []

for i in links:
    if any(i) ==  False:
        pass
    else:
        links_biden.append(str(i['web_url'])) 
len(links_biden)

2000

In [19]:
for url in links_biden:
    link = "http://api.nytimes.com/svc/community/v3/user-content/url.json?url=" + url
    payload = {'api-key': '603ff640088f24876c37e2857d83401f:1:73015248'}     
    content = single_query(link, payload)
    total = content['results']['totalCommentsFound'] 
    if total == 0:
        pass
    else:
        count = ((total/25) + 2)*25
        num_pages = list(np.arange(0, count, 25))  
            
        for i in num_pages:
            link = "http://api.nytimes.com/svc/community/v3/user-content/url.json?url=" + url
            payload = {'api-key': '60a3f7d54baade3ed03a40bdb5f5e866:3:50984754', 'offset' : i}
            print i
            for y in range(len(content['results']['comments'])): 
                new_data = content['results']['comments'][y]
                new_data['web_url'] = url
                new_data['_id'] = content['results']['comments'][y]['commentID']
                try:
                    db_comments.tab_biden.insert(new_data)
                except DuplicateKeyError:
                    print 'Peggy doesnt know why its a duplicate.'
                    pass

0
25
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
50
Peggy doesnt know

TypeError: 'NoneType' object has no attribute '__getitem__'

In [10]:
links = db.table_carson.find({},{'web_url': 1, '_id' : 0})
links_c = []

for i in links:
    print i
    if any(i) ==  False:
        pass
    else:
        links_c.append(str(i['web_url'])) 

{}
{u'web_url': u'http://www.nytimes.com/2015/09/20/opinion/sunday/ross-douthat-evangelicals-and-the-carson-illusion.html'}
{u'web_url': u'http://www.nytimes.com/aponline/2015/09/08/us/politics/ap-us-gop-2016-carson.html'}
{u'web_url': u'http://www.nytimes.com/aponline/2015/08/27/us/ap-us-gop-2016-carson.html'}
{u'web_url': u'http://query.nytimes.com/gst/abstract.html?res=9901EEDF1230E13BBC4052DFB366838D629EDE'}
{u'web_url': u'http://takingnote.blogs.nytimes.com/2015/09/02/is-ben-carson-really-the-anti-trump/'}
{u'web_url': u'http://www.nytimes.com/2015/09/14/us/politics/ben-carson-works-his-way-up-the-donation-ladder.html'}
{u'web_url': u'http://www.nytimes.com/politics/first-draft/2015/09/10/donald-trump-questions-ben-carsons-faith-and-credentials/'}
{u'web_url': u'http://www.nytimes.com/politics/first-draft/2015/09/15/today-in-politics-in-new-poll-carson-pulls-up-alongside-an-idling-trump/'}
{u'web_url': u'http://www.nytimes.com/politics/first-draft/2015/09/10/donald-trump-and-ben-c

In [14]:
for url in links_c:
    for i in range(90):
        link = "http://api.nytimes.com/svc/community/v3/user-content/url.json?url=" + url
        payload = {'api-key': '603ff640088f24876c37e2857d83401f:1:73015248', 'offset' : i}
        content = single_query(link, payload)
        if content == None:
            pass
        total = content['results']['totalCommentsFound'] 
        if total == 0:
            pass
        count = ((total/25) + 2)*25
        num_pages = list(np.arange(0, count, 25))  
        
        for y in range(len(content['results']['comments'])): 
            new_data = content['results']['comments'][y]
            new_data['web_url'] = url
            db_comments.tab_carson.insert(new_data)

KeyboardInterrupt: 